## Let's build a multimodal from scratch!
This isn't about the hype, this is truly for lovers of LLMs and the actual math behind it. If you're one of them like me, you're more than welcome to join my [discord server](https://discord.gg/PRs84AnxtT)!

### But what is a LLM anyway?

A Large Language Model (LLM) is a type of artificial intelligence model designed to understand, generate, and manipulate human language. These models are trained on vast amounts of text data and can perform a wide variety of language tasks such as translation, summarization, question answering, and text generation.

For a more in-depth understanding, check out this explanatory video:

[[ 1 hr Talk ] Intro to Large Language Models by Andrej Karpathy](https://www.youtube.com/embed/zjkBMFhNj_g)

<iframe width="560" height="315" src="https://www.youtube.com/embed/zjkBMFhNj_g" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>



#### Here's how I have divided my project structure and pipeline
```
├── models/              # Encoders & fusion layers
├── data/                # Processing scripts
├── training/            # Contrastive loss implementation
├── explainability/      # Attention visualization tools
├── demo/                # Gradio interface
```